In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pandas.api.types as ptypes
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
import optuna
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from datetime import datetime
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# activity key, value pair

activitis_dict = {"Indoor climbing" : 1, "Run" : 2, "Strength training" : 3, "Swim" : 4, "Bike" : 5, "Dancing" : 6, 
             "Stairclimber" : 7, "Spinning" : 8, "Walking" : 9, "HIIT" : 10, "Outdoor Bike" : 11, "Walk" : 12, "Aerobic Workout" : 13,
             "Tennis" : 14, "Workout" : 15, "Hike" : 16, "Zumba": 17, "Sport" : 18, "Yoga" : 19, "Swimming" : 20, "Weights" : 21,
             "Running" : 22, "Cycling" : 23, "CoreTraining" : 24}

# Dataset Processing and some explotation

In [3]:
#%%writefile dataset_preprocessing.py
class Explore_Dataset_and_Post_Processing:
    def __init__(self, train_csv_path):
        assert train_csv_path is not None, "train_csv_path cannot be none"
        self.train_csv_path = train_csv_path
        self.train_df = pd.read_csv(self.train_csv_path)
    
    # some general functions
    def get_shape(self):
        return self.train_df.shape
    
    def get_df_description(self):
        return self.train_df.describe()
    
    def get_column_wise_null(self):
        """
        return the number of values that are null
        """
        return self.train_df.isnull().sum()
    
    def get_percentage_of_missing_values(self): 
        """
        returns the pecentage of missing values
        """
        # getting the percentage of missing values
        missing_values = self.train_df.isnull().sum()
        total_rows = self.train_df.shape[0]
        missing_percentage = (missing_values / total_rows) * 100

        results_df = pd.DataFrame({'Column': missing_values.index,
                                  'Missing Values': missing_values.values,
                                  'Missing Percentage (%)': missing_percentage.values})

        results_df = results_df.sort_values(by='Missing Percentage (%)', ascending=False)
        return results_df
    
    def pp_convert_datetime(self):
        """
        converts the time column to date and time
        """
        assert ptypes.is_datetime64_any_dtype(self.train_df["time"]) is not True, "DateTime is already converted" 
        self.train_df.time = pd.to_datetime(self.train_df.time, format='%H:%M:%S')
        return self
        
    def pp_replace_null_values(self):
        """
        replace all null values with 0
        """
        # check if nan values are present
        has_nan = self.train_df.isnull().values.any()
        if not has_nan:
            print("No existing null values in the dataframe.")
        else:
            self.train_df = self.train_df.fillna(0)
        return self
            
    def pp_drop_id_pnum_column(self):
        self.train_df = self.train_df.drop(["id", "p_num", "time"], axis=1)
        return self
        
    def pp_map_activities_to_number(self, activity_dict):
        """
        Fuction to map the activity names to integers
        """
        # get all the columns with name activity
        try:
            activity_columns = list(self.train_df.filter(like='activity').columns)
            self.train_df[activity_columns] = self.train_df[activity_columns].replace(activitis_dict)
        except AssertionError as e:
            print(f"AssertionError caught: {e}")
        return self
            
    def pp_replace_nan_with_mean(self):
        """
        Function to replace the nan values with mean
        """
        # get the numerical columns in the dataset
        numerical_columns = self.train_df.select_dtypes(include = ['number']).columns
        
        # now replace for every column with nan values the  mean of that column
        self.train_df[numerical_columns] = self.train_df[numerical_columns].apply(lambda col: col.fillna(col.mean()))
        return self
        
    def pp_normalize_numerical_values(self):
        """
        Normlaize the numerical values. Here we do min ma normalization
        """
        # get the numerical columns in the dataset
        numerical_columns = list(self.train_df.select_dtypes(include = ['number']).columns)
        for column in numerical_columns:
            self.train_df[column] = (self.train_df[column] - self.train_df[column].min()) / (self.train_df[column].max() - self.train_df[column].min())
            
        return self
        
    def pp_impute_numeical_columns(self):
        numerical_columns = list(self.train_df.select_dtypes(include = ['number']).columns)
        try:
            numerical_columns.remove('bg+1:00')
        except:
            pass
        imputer = SimpleImputer(strategy='mean')
        self.train_df[numerical_columns] = imputer.fit_transform(self.train_df[numerical_columns])
        scaler = StandardScaler()
        self.train_df[numerical_columns] = scaler.fit_transform(self.train_df[numerical_columns])
        return self
        
    def pp_encode_categorical_columns(self):
        categorical_cols = [col for col in self.train_df.columns if 'activity' in col]
        for col in categorical_cols:
            self.train_df[col] = self.train_df[col].fillna('None')
            le = LabelEncoder()
            self.train_df[col] = le.fit_transform(self.train_df[col])
        return self

    def pp_drop_columns_with_missing_values(self, missing_df,pecentage):
        """
        missing_value_df : o/p of the function obtained from "get_percentage_of_missing_values"
        percentage : column with missing values >= pecentage is dropped for training and testing
        """
        # list of columns with missing values  > 90
        filtered_columns = missing_df[missing_df['Missing Percentage (%)'] > pecentage]['Column'].tolist()
        # now drop all these columns
        self.train_df = self.train_df.drop(filtered_columns, axis = 1)
        return self

    def get_final_df(self):
        return self.train_df
            

# Unit Tests if needed 

In [ ]:
class Unit_Tests:
    def __init__(self, df):
        self.df = df
        
    def check_nan_replaced_with_mean(self):
        # first check no Nan values
        numerical_columns = self.df.select_dtypes(include = ['number']).columns
        for col in numerical_columns:
            if self.df[col].isna().sum() == 0:
                return True
            else:
                return False

        
    def check_values_are_normalized(self):
        # check if values are normalized
        numerical_columns = self.df.select_dtypes(include = ['number']).columns
        for col in numerical_columns:
            has_greater_than_one = (self.df[col] > 1).any()
            if has_greater_than_one:
                return False
            
        return True

In [ ]:
## This part will be the Unit tests column
train_csv_path = "/kaggle/input/brist1d/train.csv"
obj_ex = Explore_Dataset_and_Post_Processing(train_csv_path)
obj_ex.pp_normalize_numerical_values()

obj_utest = Unit_Tests(df = obj_ex.train_df)
obj_utest.check_values_are_normalized()

# Dataset Creation

In [4]:
class DatasetCreation:
    def __init__(self, train_df, activities_dict, train_ratio, test_ratio):
        self.actvities_dict = activities_dict
        """
        few assertions and checks needed to be done
        """
        # check if all nun values are non eistent
        if train_df.isnull().values.any():
            raise ValueError("There should not be NaN in training data")
        
        # assert that id and p_num is dropped
        columns = list(train_df.columns)
        assert "id" not in columns, "There should not be the id column"
        assert "p_num" not in columns, "There should not be the p_num column"
        
        # check that all the activities column have dtype float64
        activity_columns = list(train_df.filter(like='activity').columns)
        for col in activity_columns:
            for value in train_df[col]:
                if value == 0:
                    pass
                elif value not in self.actvities_dict.values():
                    raise AssertionError(f"Value {value} in column {col} was not correctly mapped!")
        
        # check if train and test ratio is float
        assert isinstance(train_ratio, float), f"Expected float, but got {type(x).__name__}"
        assert isinstance(test_ratio, float), f"Expected float, but got {type(x).__name__}"
        
        self.train_df = train_df
        self.train_ratio = train_ratio
        self.test_ratio = test_ratio
        
    def return_dataset(self):
        train_X = self.train_df.drop(['bg+1:00'], axis = 1)
        train_Y = self.train_df['bg+1:00']
        X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(train_X, train_Y, test_size=self.test_ratio, random_state=42)
        
        return X_train_split, X_val_split, y_train_split, y_val_split

In [5]:
train_csv_path = "/kaggle/input/brist1d/train.csv"
obj_data_pipeline = Explore_Dataset_and_Post_Processing(train_csv_path)

# Simple training Data creation pipeline
"""
Here the order of preprocessing is very important
"""
# get the percentage of missing values df
missing_values_df = obj_data_pipeline.get_percentage_of_missing_values()
print(missing_values_df.head())
# preporcessing data
processed_train_df = (obj_data_pipeline
                     .pp_drop_id_pnum_column()
                     .pp_drop_columns_with_missing_values(missing_values_df, 90)
                     .pp_replace_null_values()
                     .pp_map_activities_to_number(activitis_dict)
                     .get_final_df())

print("[!] Number of columns in processed df : ", len(processed_train_df.columns))
obj_data_create = DatasetCreation(processed_train_df, activities_dict=activitis_dict, train_ratio=0.8, test_ratio=0.2)
x_train, x_val, y_train, y_val  = obj_data_create.return_dataset()

print("Type of X_train : ", type(x_train))

/tmp/ipykernel_31/735429745.py:6: DtypeWarning: Columns (435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506) have mixed types. Specify dtype option on import or set low_memory=False.
  self.train_df = pd.read_csv(self.train_csv_path)


         Column  Missing Values  Missing Percentage (%)
164  carbs-4:30          174496               98.571945
170  carbs-4:00          174492               98.569685
161  carbs-4:45          174491               98.569121
155  carbs-5:15          174490               98.568556
152  carbs-5:30          174490               98.568556
[!] Number of columns in processed df :  361
Type of X_train :  <class 'pandas.core.frame.DataFrame'>


# XGBOOST 

## Using Optuna to do hyper parameter Tuning

In [6]:
"""
Using optuna for hyperparameter tuning
## Refrence : https://www.kaggle.com/code/danishyousuf19/blood-glucose-prediction
"""
def objective(trial):
    # Suggest hyperparameters for tuning
    params = {
        'objective': 'reg:squarederror',
        'eval_metric': 'rmse',
        'booster': 'gbtree',
        'n_estimators': trial.suggest_int('n_estimators', 50, 1500),
        'eta': trial.suggest_loguniform('eta', 0.0005, 0.3),  # learning rate
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'min_child_weight': trial.suggest_loguniform('min_child_weight', 1e-4, 10),
        'subsample': trial.suggest_uniform('subsample', 0.3, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.09, 1.0),
        'lambda': trial.suggest_loguniform('lambda', 3, 10),
        'alpha': trial.suggest_loguniform('alpha', 1e-4, 10),
        'tree_method': 'hist',  
        'device':'cuda'
    }
    dtrain = xgb.DMatrix(x_train, label=y_train)
    dvalid = xgb.DMatrix(x_val, label=y_val)
    
    # Train the model
    model = xgb.train(params, dtrain, evals=[(dvalid, 'validation')], num_boost_round=1500, early_stopping_rounds=35, verbose_eval=False)
    
    # Predict on the validation set
    y_pred_valid = model.predict(dvalid)
    
    # Calculate RMSE on the validation set
    rmse = mean_squared_error(y_val, y_pred_valid, squared=False)
    print("RMSE = ",rmse)
    return rmse

In [7]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=10)
best_params = study.best_params
print(f"Best hyperparameters: {best_params}")



[I 2024-10-24 19:17:00,671] A new study created in memory with name: no-name-3cd6e28c-9b7d-43dc-9d79-4b5a308d4790
/tmp/ipykernel_31/1999087542.py:12: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'eta': trial.suggest_loguniform('eta', 0.0005, 0.3),  # learning rate
/tmp/ipykernel_31/1999087542.py:14: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'min_child_weight': trial.suggest_loguniform('min_child_weight', 1e-4, 10),
/tmp/ipykernel_31/1999087542.py:15: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_un

RMSE =  2.024354083763068


/tmp/ipykernel_31/1999087542.py:12: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'eta': trial.suggest_loguniform('eta', 0.0005, 0.3),  # learning rate
/tmp/ipykernel_31/1999087542.py:14: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'min_child_weight': trial.suggest_loguniform('min_child_weight', 1e-4, 10),
/tmp/ipykernel_31/1999087542.py:15: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample', 0.3, 1.0),
/tmp/ipykernel_31/1999087542.py:16: FutureWarning: suggest_uniform has been deprecat

RMSE =  1.6716266417045211


/tmp/ipykernel_31/1999087542.py:12: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'eta': trial.suggest_loguniform('eta', 0.0005, 0.3),  # learning rate
/tmp/ipykernel_31/1999087542.py:14: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'min_child_weight': trial.suggest_loguniform('min_child_weight', 1e-4, 10),
/tmp/ipykernel_31/1999087542.py:15: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample', 0.3, 1.0),
/tmp/ipykernel_31/1999087542.py:16: FutureWarning: suggest_uniform has been deprecat

...

RMSE =  1.4374974943251968


/tmp/ipykernel_31/1999087542.py:12: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'eta': trial.suggest_loguniform('eta', 0.0005, 0.3),  # learning rate
/tmp/ipykernel_31/1999087542.py:14: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'min_child_weight': trial.suggest_loguniform('min_child_weight', 1e-4, 10),
/tmp/ipykernel_31/1999087542.py:15: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample', 0.3, 1.0),
/tmp/ipykernel_31/1999087542.py:16: FutureWarning: suggest_uniform has been deprecat

RMSE =  2.046590236099163


/tmp/ipykernel_31/1999087542.py:12: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'eta': trial.suggest_loguniform('eta', 0.0005, 0.3),  # learning rate
/tmp/ipykernel_31/1999087542.py:14: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'min_child_weight': trial.suggest_loguniform('min_child_weight', 1e-4, 10),
/tmp/ipykernel_31/1999087542.py:15: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample', 0.3, 1.0),
/tmp/ipykernel_31/1999087542.py:16: FutureWarning: suggest_uniform has been deprecat

RMSE =  1.5965784107519423
Best hyperparameters: {'n_estimators': 365, 'eta': 0.020093639162481303, 'max_depth': 14, 'min_child_weight': 0.005971759564101484, 'subsample': 0.6261651700121292, 'colsample_bytree': 0.11624704554396792, 'lambda': 8.533782245808666, 'alpha': 1.6511090194768308}


In [8]:
## Defining the best params obtained from the previous step
best_params = {
    'n_estimators': 365,
    'eta': 0.020093639162481303,
    'max_depth': 14,
    'min_child_weight': 0.005971759564101484,
    'subsample': 0.62616517001,
    'colsample_bytree': 0.11624704554396792,
    'lambda': 8.533782245808666, 
    'alpha': 1.6511090194768308
}

## Class creating the XGBoost Model
### Depending on the best params received above

In [9]:
class Model_XGBoost:
    def __init__(self, params, X, Y, random_state,
                 objective = 'reg:squarederror', 
                 eval_metric = 'rmse', 
                 device = 'cuda', 
                 tree_method = 'hist',
                 booster = 'gbtree'):
        """
        X : training data X
        Y : training data Y
        params : dictionary of paramteres that were predicted above
        """
        # Train an XGBoost model
        params["objective"] = objective
        params["eval_metric"] = eval_metric
        params["device"] = device
        params["tree_method"] = tree_method
        params["booster"] = booster
        self.params = params
        self.objective = objective
        self.n_estimators = n_estimators
        self.random_state = random_state
        self.X = X
        self.Y = Y
    
    def get_xgb_model(self):
        """
        Using the XGBoost model directly with paramteres we got from optuna
        """
        dtrain = xgb.DMatrix(self.X, label=self.Y)
        model = xgb.train(self.params, dtrain, num_boost_round=1500)
        return model
        
    def get_Regressor_model(self):
        xgb_model = XGBRegressor(objective = self.objective,
                                      n_estimators = self.n_estimators, 
                                      random_state = self.random_state)
        return xgb_model
    
    def fit(self):
        self.xgb_model.fit(self.X, self.Y)

    def xgb_predictions(self, model, x_val = None):
        assert x_val is not None, "Validation data cannot be None"
        xgb_predictions = model.predict(x_val)
        return xgb_predictions
    
    def xgb_rmse(self, y_val, predictions):
        xgb_rmse = np.sqrt(mean_squared_error(y_val, predictions))
        return xgb_rmse

In [10]:
"""
Class to implement cross validation incase it is needed
"""
from sklearn.model_selection import cross_val_score, KFold
class CrossValidation:
    def __init__(self, num_of_splits):
        self.kf = KFold(n_splits=num_of_splits, shuffle=True, random_state=42)
    def get_cv(self):
        return self.kf

## Training and prediction

In [12]:
# here is the experiment to see how number of estimators effect the RMSE
n_estimators = [100]
rmse = []
time_taken = []
implement_cross_validation = False

if implement_cross_validation == False:
    print("[*] Going for Normal XGBOOST")
    #for i in n_estimators:
    start_time = datetime.now()
    obj_xgb_model = Model_XGBoost(best_params,
                             random_state= 42,
                             X = x_train,
                             Y = y_train)

    #xgb_model.fit()
    model = obj_xgb_model.get_xgb_model()
    d_val = xgb.DMatrix(x_val)
    predictions = model.predict(d_val)
    #rmse.append(obj_xgb_model.xgb_rmse(y_val, predictions))
    rmse = obj_xgb_model.xgb_rmse(y_val, predictions)
    print("RMSE : ", rmse)
    end_time = datetime.now()
    #time_taken.append(end_time - start_time)
    print("Total time taken : ", end_time - start_time)
else:
    print("[*] Going fo Cross validation")
    
    start_time = datetime.now()
    obj_xg = Model_XGBoost(objective= 'reg:squarederror',
                             n_estimators= i,
                             random_state= 42,
                             X = x_train,
                             Y = x_train)

    boost_model = obj_xg.get_model()

    obj_cv = CrossValidation(num_of_splits=5)
    kf = obj_cv.get_cv()
    
    for train_index, validation_index in kf.split(x_train):
        X_train, X_val = x_train.iloc[train_index], x_train.iloc[validation_index]
        Y_train, Y_val = y_train.iloc[train_index], y_train.iloc[validation_index]
        boost_model.fit(X_train, Y_train)
        models.append(boost_model)
        predictions = boost_model.predict(x_val)
        rmse = obj_xg.xgb_rmse(y_val, predictions)
        val_predictions_rmse.append(rmse)
        print("Prediction : ", rmse)
    end_time = datetime.now()
    time_taken.append(end_time - start_time)
    

[*] Going for Normal XGBOOST


/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [20:10:04] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


RMSE :  1.4067348589586193
Total time taken :  0:02:30.511217


In [13]:
# saveing the model
model.save_model("/kaggle/working/xgb_best_model_24_10_2024.bin")

/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [20:18:41] WARNING: /workspace/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)


In [ ]:
# some utility stuff for plotting
import matplotlib.pyplot as plt
# Plotting RMSE vs n_estimators
plt.figure(figsize=(14, 6))

# First subplot for RMSE vs n_estimators
plt.subplot(1, 2, 1)
plt.plot(n_estimators, rmse, marker='o', color='b')
plt.title('RMSE vs Number of Estimators')
plt.xlabel('Number of Estimators')
plt.ylabel('RMSE')
plt.grid(True)

# Second subplot for Time taken vs n_estimators
time_taken_in_seconds = [t.total_seconds() for t in time_taken]
plt.subplot(1, 2, 2)
plt.plot(n_estimators, time_taken_in_seconds, marker='o', color='r')
plt.title('Time Taken vs Number of Estimators')
plt.xlabel('Number of Estimators')
plt.ylabel('Time Taken (seconds)')
plt.grid(True)

# Show the plots
plt.tight_layout()
plt.show()

### From the above graph we can observe the following
1. RMSE decreases non-linearly with the increase in the number of estimators
2. Time increases linearly with the number of estimators

# Test And Submission 

Here this is just for my test and finalizing which model will I be using.

I am using a separate notebook for submission

In [ ]:
final_xgb_model = Model_XGBoost(objective= 'reg:squarederror',
                             n_estimators= 700,
                             random_state= 42,
                             X = x_train,
                             Y = y_train)
final_xgb_model.fit()
final_xgb_model.xgb_model.save_model("/kaggle/working/xgb_est_700_20_10_2024.bin")

In [ ]:
# load_submission_df
submission_df = pd.read_csv("/kaggle/input/brist1d/sample_submission.csv")

# load test_csv
test_csv_path = "/kaggle/input/brist1d/test.csv"
obj_ex_test = Explore_Dataset_and_Post_Processing(test_csv_path)

# Simple training Data creation pipeline
obj_ex_test.pp_convert_datetime()
obj_ex_test.pp_drop_id_pnum_column()
#obj_ex.pp_replace_nan_with_mean()
#obj_ex.pp_normalize_numerical_values()
#obj_ex.pp_replace_null_values()
#obj_ex.pp_map_activities_to_number(activitis_dict)
obj_ex_test.pp_impute_numeical_columns()
obj_ex_test.pp_encode_categorical_columns()
obj_ex_test.pp_replace_null_values()

# Choose the best model based on validation performance and make predictions on the test set

test_predictions = xgb_model.predict(obj_ex_test.train_df)

# Create a submission DataFrame
submission_df['bg+1:00'] = test_predictions

# Save the submission file
submission_df.to_csv('submission.csv', index=False)



## The 700 model  is not helping at all